In [27]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

def parse_text(unparsed):
    parsed = unparsed.get_text().lstrip('\n\r\n').lstrip('\n\r\n ').rstrip('\n\r\n').rstrip('\n\r\n ')
    return parsed

def extract_uuid(string):
    uuid_regex = re.compile("[0-9a-f]{8}-[0-9a-f]{4}-[0-9a-f]{4}-[0-9a-f]{4}-[0-9a-f]{12}")
    return re.search(uuid_regex, string).group(0)
    
def remove_non_numberic(string):
    string = string.strip().replace(",", "")
    return re.sub("[^0-9]", "", string)

In [28]:
cazoo_page = requests.get("https://www.cazoo.co.uk/cars/mercedes-benz/a-class/?colour=Grey&fuelType=Petrol")

In [29]:
cazoo_soup = BeautifulSoup(cazoo_page.content, 'html.parser')
cazoo_search_results = cazoo_soup.find_all("div", {"class" : re.compile('.*vehicle-cardstyles__Card-sc.*')})

In [34]:
#Cazoo Listings Dictionary

cazoo_listings = []

In [35]:
#Cazoo Listings

for s in cazoo_search_results:
    
    car_id = extract_uuid(s.find('a', href=True)['href'])

    details = s.findAll("div", {"class" : re.compile('.*vehicle-cardstyles__DetailWrap-sc.*')})
    
    for d in details:
        tags = d.find_next('ul').find_all('li')

        title = parse_text(d.find("p", {"class": re.compile('.*vehicle-cardstyles__Title-sc.*')}))
        description = parse_text(d.find("p", {"class" : re.compile('.*vehicle-cardstyles__DisplayVariant-sc.*')}))
        mileage = remove_non_numberic(parse_text(tags[0]))
        year = remove_non_numberic(parse_text(tags[1]))
        transmission = parse_text(tags[2])
        fuel = parse_text(tags[3])

    price_gbp = remove_non_numberic(parse_text(s.find("p", {"data-test-id": "card-pricing-full-price-gb"})))

    listing_details = { 'car_id': car_id,
                        'car_title': title,
                        'car_decsription': description,
                        'mileage': mileage,
                        'year': year,
                        'transmission': transmission,
                        'fuel': fuel,
                        'price_gbp': price_gbp,
                        'source': 'cazoo',
                        'url': f"https://www.cazoo.co.uk/car-details/{car_id}/"
                        }
    
    cazoo_listings.append(listing_details)


In [39]:
# DataFrame
df = pd.DataFrame(cazoo_listings)
df.head()

,car_id,car_title,car_decsription,mileage,year,transmission,fuel,price_gbp,source,url
0,6a6c65ae-5dff-54dc-a011-14e74ef1a9e8,Mercedes-Benz A Class,1.3L SE A180,23532,2019,Manual,Petrol,18475,cazoo,https://www.cazoo.co.uk/car-details/6a6c65ae-5...
1,3d0f6b3d-7703-5310-9a72-e23e3f6fc5b7,Mercedes-Benz A Class,1.3L SE A180,22971,2019,Manual,Petrol,18900,cazoo,https://www.cazoo.co.uk/car-details/3d0f6b3d-7...
2,421a7618-3a4f-5c7c-932d-55f1cc981990,Mercedes-Benz A Class,1.3L SE A180,26920,2020,Manual,Petrol,18925,cazoo,https://www.cazoo.co.uk/car-details/421a7618-3...
3,cb3f0617-fddf-5af1-a697-4824bb9ef1aa,Mercedes-Benz A Class,1.6L AMG Line A180,19406,2016,Automatic,Petrol,18200,cazoo,https://www.cazoo.co.uk/car-details/cb3f0617-f...
4,202c6763-4214-5129-8567-96cd4c14846a,Mercedes-Benz A Class,1.3L SE A180,8127,2019,Manual,Petrol,20150,cazoo,https://www.cazoo.co.uk/car-details/202c6763-4...
